# XOR with tensorflow

In [ ]:
import tensorflow as tf
import numpy as np
import os

In [ ]:
xy = np.loadtxt('train.txt', unpack=True)
x_data_source = xy[0:-1]
y_data_source = xy[-1]

## single layer models

### model

In [ ]:
x_data = x_data_source
y_data = y_data_source

print(x_data)

with tf.name_scope("SLP"):
    with tf.name_scope("model"):
        X = tf.placeholder(tf.float32, [3, None], name = 'input')
        Y = tf.placeholder(tf.float32, name = 'expect')
        W = tf.Variable(tf.random_uniform([1, x_data.shape[0]], -1.0, 1.0), name = 'weight-bias')
        Y_ = tf.sigmoid(tf.matmul(W, X), name = 'predict')
        tf.summary.histogram("weight-bias", W)

    with tf.name_scope("update"):
        cost = - tf.reduce_mean(Y * tf.log(Y_) + (1-Y)*tf.log(1-Y_), name = 'error')
        alpha = 0.1
        optimizer = tf.train.GradientDescentOptimizer(alpha)
        train = optimizer.minimize(cost)
        tf.summary.scalar('cost', cost)
        embedding_var = tf.Variable(tf.zeros([1, x_data.shape[1]]), name = 'embd')
        embedding_var.assign(Y_) # 1D matrix can't be seen on tensorboard

    with tf.name_scope("evaluate"):
        correct_prediction = tf.equal(tf.floor(Y_ + 0.5), Y)
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))
        tf.summary.scalar("accuracy", accuracy)

## multilayer models

### multilayer model 1 - row vector

```
Y [ 0.  1.  1.  0.]
Y shape <unknown>
log y [[-0.79689521]
 [-0.7082116 ]
 [-0.72524583]
 [-0.65353537]]
log y shape (?, 1)
```

so Y * log(y\_) = [4, 4]

In [ ]:
x_data = x_data_source.transpose()
y_data = y_data_source.transpose()

print(x_data)

with tf.name_scope("MLP"):
    with tf.name_scope("layer1"):
        X = tf.placeholder(tf.float32, [None, 3], name = 'input')
        Y = tf.placeholder(tf.float32, name = 'output')
        W1 = tf.Variable(tf.random_uniform([3, 2], -1.0, 1.0), name = 'weight-bias-1')
        l1 = tf.sigmoid(tf.matmul(X, W1), name = 'layer1')
        tf.summary.histogram("weight-bias-1", W1)

    with tf.name_scope("layer2"):
        W2 = tf.Variable(tf.random_uniform([2, 1], -1.0, 1.0), name = 'weight-2')
        b2 = tf.Variable(tf.random_uniform([1], -1.0, 1.0), name = 'bias-2')
        Y_ = tf.sigmoid(tf.matmul(l1, W2) + b2, name = 'prediction')
        tf.summary.histogram("weight-2", W2)
        tf.summary.histogram("bias-2", b2)
        tf.summary.histogram("prediction", Y_)
        embedding_var = tf.Variable(tf.zeros([x_data.shape[0], 2]), name = 'embd')
        embedding_var.assign(l1)

    with tf.name_scope("update"):
        # caution
        # y_: [?, 1], Y: [1, ?]
        # Y * y_: [num of data , num of data]
        # Y * transpose(y_): [1, num of data]

        cost = - tf.reduce_mean(Y * tf.transpose(tf.log(Y_)) + (1-Y) * tf.transpose(tf.log(1 - Y_)), name = 'cost')
        alpha = 0.1
        optimizer = tf.train.GradientDescentOptimizer(alpha)
        train = optimizer.minimize(cost)
        tf.summary.scalar('cost', cost)



### multilayer model 2 - column vector

```
Y [ 0.  1.  1.  0.]
Y shape <unknown>
log y [[-0.86359024 -0.86644518 -0.86220801 -0.86514658]]
log y shape (1, ?)
```

so Y * log(y\_) = [1, 4]

In [ ]:
x_data = x_data_source
y_data = y_data_source

print(x_data)

with tf.name_scope("MLP"):
    with tf.name_scope("layer1"):
        X = tf.placeholder(tf.float32, [3, None], name = 'input')
        Y = tf.placeholder(tf.float32, name = 'output')
        W1 = tf.Variable(tf.random_uniform([2, 3], -1.0, 1.0), name = 'weight-bias-1')
        l1 = tf.sigmoid(tf.matmul(W1, X), name = 'layer-1')
        tf.summary.histogram("weight-bias-1", W1)
        
    with tf.name_scope("layer2"):
        W2 = tf.Variable(tf.random_uniform([1, 2], -1.0, 1.0), name = 'weight-2')
        b2 = tf.Variable(tf.random_uniform([1], -1.0, 1.0), name = 'bias-2')
        Y_ = tf.sigmoid(tf.matmul(W2, l1) + b2, name = 'prediction')
        tf.summary.histogram("weight-2", W2)
        tf.summary.histogram("bias-2", b2)
        tf.summary.histogram("prediction", Y_)
        embedding_var = tf.Variable(tf.zeros([x_data.shape[1], 2]), name = 'embd')
        embedding_var.assign(tf.transpose(l1))

    with tf.name_scope("update"):
        cost = - tf.reduce_mean(Y * tf.log(Y_) + (1 - Y) * tf.log(1-Y_), name = 'cost')
        alpha = 0.1
        optimizer = tf.train.GradientDescentOptimizer(alpha)
        train = optimizer.minimize(cost)
        tf.summary.scalar('cost', cost)

### wide model

In [ ]:
x_data = x_data_source
y_data = y_data_source

print(x_data)

with tf.name_scope("MLP-wide"):
    with tf.name_scope("layer1"):
        X = tf.placeholder(tf.float32, [3, None], name = 'input')
        Y = tf.placeholder(tf.float32, name = 'output')
        W1 = tf.Variable(tf.random_uniform([10, 3], -1.0, 1.0), name = 'weight-bias-1')
        l1 = tf.sigmoid(tf.matmul(W1, X), name = 'layer-1')
        tf.summary.histogram("weight-bias-1", W1)
        
    with tf.name_scope("layer2"):
        W2 = tf.Variable(tf.random_uniform([1, 10], -1.0, 1.0), name = 'weight-2')
        b2 = tf.Variable(tf.random_uniform([1], -1.0, 1.0), name = 'bias-2')
        Y_ = tf.sigmoid(tf.matmul(W2, l1) + b2, name = 'prediction')
        tf.summary.histogram("weight-2", W2)
        tf.summary.histogram("bias-2", b2)
        tf.summary.histogram("prediction", Y_)
        embedding_var = tf.Variable(tf.zeros([x_data.shape[1], 10]), name = 'embd')
        embedding_var.assign(tf.transpose(l1))

    with tf.name_scope("update"):
        cost = - tf.reduce_mean(Y * tf.log(Y_) + (1 - Y) * tf.log(1-Y_), name = 'cost')
        alpha = 0.1
        optimizer = tf.train.GradientDescentOptimizer(alpha)
        train = optimizer.minimize(cost)
        tf.summary.scalar('cost', cost)


### deep model

In [ ]:
x_data = x_data_source
y_data = y_data_source

print(x_data)

with tf.name_scope("MLP-wide"):
    with tf.name_scope("layer1"):
        X = tf.placeholder(tf.float32, [3, None], name = 'input')
        Y = tf.placeholder(tf.float32, name = 'output')
        W1 = tf.Variable(tf.random_uniform([6, 3], -1.0, 1.0), name = 'weight-bias-1')
        l1 = tf.sigmoid(tf.matmul(W1, X), name = 'layer-1')
        tf.summary.histogram("weight-bias-1", W1)
        
    with tf.name_scope("layer2"):
        W2 = tf.Variable(tf.random_uniform([2, 6], -1.0, 1.0), name = 'weight-2')
        b2 = tf.Variable(tf.random_uniform([2], -1.0, 1.0), name = 'bias-2')
        
        # broadcast bias. 
        #l2 = tf.sigmoid(tf.matmul(W2, l1) + b2, name = 'layer-2') # l2 -> (2, 2)
        #l2 = tf.transpose(tf.sigmoid(tf.transpose(tf.matmul(W2, l1)) + b2, name = 'layer-2')) # l2 -> (2, ?) but trx, trx, ...
        b2 = tf.reshape(b2, [2, -1]) # l2 -> (2, -1)
        l2 = tf.sigmoid(tf.matmul(W2, l1) + b2, name = 'layer-2')
        tf.summary.histogram("weight-2", W2)
        tf.summary.histogram("bias-2", b2)
        tf.summary.histogram("layer-2", l2)

    with tf.name_scope("layer3"):
        W3 = tf.Variable(tf.random_uniform([1, 2], -1.0, 1.0), name = 'weight-3')
        b3 = tf.Variable(tf.random_uniform([1], -1.0, 1.0), name = 'bias-3')
        Y_ = tf.sigmoid(tf.matmul(W3, l2) + b3, name = 'prediction')
        tf.summary.histogram("weight-3", W3)
        tf.summary.histogram("bias-3", b3)
        tf.summary.histogram("prediction", Y_)
        embedding_var = tf.Variable(tf.zeros([x_data.shape[1], 2]), name = 'embd')
        embedding_var.assign(tf.transpose(l2))

    with tf.name_scope("update"):
        cost = - tf.reduce_mean(Y * tf.log(Y_) + (1 - Y) * tf.log(1-Y_), name = 'cost')
        alpha = 0.1
        optimizer = tf.train.GradientDescentOptimizer(alpha)
        train = optimizer.minimize(cost)
        tf.summary.scalar('cost', cost)
        

## evaluate

In [ ]:
with tf.name_scope("Evaluate"):
    correct_prediction = tf.equal(tf.floor(Y_ + 0.5), Y)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'), name = 'accuracy')
    tf.summary.scalar("accuracy", accuracy)

## learning

In [ ]:
%rm -rf xor_log

In [ ]:
summary_op = tf.summary.merge_all()
writer = tf.summary.FileWriter("xor_log")
saver = tf.train.Saver()

In [ ]:
# embedding metadata - link label file
from tensorflow.contrib.tensorboard.plugins import projector
config = projector.ProjectorConfig()

embedding = config.embeddings.add()
embedding.tensor_name = embedding_var.name
embedding.metadata_path = os.path.join("./", 'metadata.tsv')

projector.visualize_embeddings(writer, config)

In [ ]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    writer.add_graph(sess.graph)
    for step in range(10000):
        if (step % 1000 == 0):
            run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
            run_metadata = tf.RunMetadata()
            summary, cost_val, _ = sess.run([summary_op, cost, train], feed_dict = {X:x_data, Y:y_data}, options = run_options, run_metadata=run_metadata)
            writer.add_run_metadata(run_metadata, 'step%d' %step)
            sess.run([embedding_var], feed_dict = {X:x_data})
            print("step:", step, "cost:", cost_val)
            saver.save(sess, os.path.join("xor_log", "model.ckpt"), step)
        else:
            summary, cost_val, _ = sess.run([summary_op, cost, train], feed_dict = {X:x_data, Y:y_data})
        writer.add_summary(summary, step)
    print("accuracy:", sess.run([accuracy], feed_dict = {X:x_data, Y:y_data}))



In [ ]:
!tensorboard --log=xor_log